In [ ]:
# V16_unsup recap -> moved to unsup learning to unsup transfer learning
    # config section:  swapped SSL for V13 model
    # section #1:  added torch.device
    # section #3:  added outputs = model(**inputs)
        # added if hasattr(outputs, "pooler_output")

In [40]:
# ========== IMPORTS SECTION ==========

# ===== STANDARD LIBRARY =====
import datetime
import glob
import os
import platform
import shutil
import time

# ===== THIRD PARTY: CORE =====
import numpy as np
import pandas as pd
import psutil

# ===== VISUALIZATION =====
import matplotlib.pyplot as plt
import seaborn as sns

# ===== ML/DEEP LEARNING =====
import torch
import torch.nn as nn
import torch.optim as optim

# ===== IMAGE/PROCESSING =====
from PIL import Image
from scipy.stats import entropy
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm import tqdm

# ===== TRANSFORMERS =====
from transformers import AutoImageProcessor, AutoModel, ViTForImageClassification

# ===== OPTIONAL: UMAP =====
try:
    import umap
    HAVE_UMAP = True
except ImportError:
    HAVE_UMAP = False

In [22]:
# ========== CONFIG SECTION (Version-controlled) ==========
CFG_VERSION = "V16_unsup_2025_06_04"
V13_MODEL_PATH = "V13_20250527_161430"
V13_PROCESSOR_PATH = V13_MODEL_PATH
UNSUP_BASE_DIR = "/Users/natalyagrokh/AI/ml_expressions/img_expressions_unsup"
IMAGE_DIR = "/Users/natalyagrokh/AI/ml_expressions/img_datasets/temp_autogluon_images"
DATASET_NAME = os.path.basename(os.path.normpath(IMAGE_DIR))
DATESTAMP = datetime.datetime.now().strftime("%Y%m%d")

N_CLUSTERS = 20
KL_OOD_THRESHOLD = 0.55
EUCLIDEAN_OUTLIER_PERCENTILE = 98
BATCH_SIZE = 32
VALID_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.webp', '.jfif')
T_SNE_SUBSAMPLE = 5000
CLUSTER_N_INIT = 10

# Auto-versioned output filenames
OUT_PREFIX = f"{CFG_VERSION}_{DATASET_NAME}_clusters{N_CLUSTERS}_kl{KL_OOD_THRESHOLD}_eu{EUCLIDEAN_OUTLIER_PERCENTILE}_{DATESTAMP}"

#define label names
LABEL_NAMES = [
    "anger", "disgust", "fear", "happiness",
    "sadness", "surprise", "neutral", "questioning"
]
id2label = dict(enumerate(LABEL_NAMES))
label2id = {v: k for k, v in id2label.items()}

In [3]:
# ======= DYNAMIC UNSUPERVISED OUTPUT FOLDER CREATION ========
def get_next_unsup_version(base_dir=".", prefix="V"):
    all_entries = glob.glob(os.path.join(base_dir, f"{prefix}*_*"))
    existing = [
        os.path.basename(d) for d in all_entries if os.path.isdir(d)
    ]
    versions = [
        int(d[1:].split("_")[0]) for d in existing
        if d.startswith(prefix) and "_" in d and d[1:].split("_")[0].isdigit()
    ]
    next_version = max(versions, default=0) + 1
    return f"{prefix}{next_version}"

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
VERSION = get_next_unsup_version(base_dir=".")
VERSION_TAG = VERSION + "_unsup_" + timestamp
SAVE_DIR = os.path.join(".", VERSION_TAG)
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"📁 Unsupervised output directory created: {SAVE_DIR}")

📁 Unsupervised output directory created: ./V16_unsup_20250604_142640


In [4]:
# ========== ARTIFACT PREFIXES ==========
OUT_PREFIX = os.path.join(
    SAVE_DIR, f"{VERSION_TAG}_{DATASET_NAME}_clusters{N_CLUSTERS}_kl{KL_OOD_THRESHOLD}_eu{EUCLIDEAN_OUTLIER_PERCENTILE}"
)
EMBED_FILE = f"{OUT_PREFIX}_embeddings.npy"
PATHS_FILE = f"{OUT_PREFIX}_paths.npy"

In [5]:
# ========== HARDWARE/ENV LOG ==========
print("===== SYSTEM INFO =====")
print("Python:", platform.python_version())
print("Platform:", platform.platform())
print("CPU:", platform.processor())
print("RAM (GB):", round(psutil.virtual_memory().total / 1e9, 2))
print("=======================")
print("🖥️ Using CPU only for feature extraction (edit if using GPU)")

===== SYSTEM INFO =====
Python: 3.10.16
Platform: macOS-15.5-arm64-arm-64bit
CPU: arm
RAM (GB): 17.18
🖥️ Using CPU only for feature extraction (edit if using GPU)


In [6]:
# ========== 1. LOAD SUPERVISED (V13) MODEL ==========
print(f"🔄 Loading V13 backbone: {V13_MODEL_PATH}")
device = torch.device("cpu")  # Change to "cuda" if using GPU
model = ViTForImageClassification.from_pretrained(V13_MODEL_PATH, local_files_only=True).to(device).eval()
processor = AutoImageProcessor.from_pretrained(V13_MODEL_PATH, local_files_only=True)

# Print model info
print("✅ Model config loaded:")
print(f"  Model arch: {model.config.architectures}")
print(f"  # Labels: {model.config.num_labels}")
if hasattr(model.config, "id2label"):
    print(f"  id2label: {model.config.id2label}")
print("✅ Processor loaded:", type(processor).__name__)
print("🖥️ Model device:", model.device)
model = model.to("cpu").eval()  # Or use CUDA/MPS as available

Some weights of the model checkpoint at V13_20250527_161430 were not used when initializing ViTForImageClassification: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at V13_20250527_161430 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔄 Loading V13 backbone: V13_20250527_161430
✅ Model config loaded:
  Model arch: ['ViTForImageClassification']
  # Labels: 8
  id2label: {0: 'anger', 1: 'disgust', 2: 'fear', 3: 'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral', 7: 'questioning'}
✅ Processor loaded: ViTImageProcessorFast
🖥️ Model device: cpu


In [7]:
# ========== 2. IMAGE PATHS ==========
image_paths = [
    os.path.join(IMAGE_DIR, f)
    for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith(VALID_EXTENSIONS)
]
print(f"🖼️ Found {len(image_paths)} images.")

🖼️ Found 31002 images.


In [8]:
# # ========== LOADING SAVED NPY FILES WHEN CODE FAILS ==========

# # Adjust these to your actual saved file locations (if you use dynamic folders, fill in SAVE_DIR and OUT_PREFIX accordingly)
# EMBED_FILE = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/V16_unsup_2025_06_04_temp_autogluon_images_clusters20_kl0.55_eu98_20250604_embeddings.npy"  # e.g., "/Users/natalyagrokh/AI/ml_expressions/img_expressions_unsup/V16_unsup_20250604_191044/V16_unsup_20250604_191044_temp_autogluon_images_clusters20_kl0.55_eu98_embeddings.npy"
# PATHS_FILE = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/V16_unsup_2025_06_04_temp_autogluon_images_clusters20_kl0.55_eu98_20250604_paths.npy"

# # Load arrays
# all_embeddings = np.load(EMBED_FILE)
# all_files = np.load(PATHS_FILE, allow_pickle=True).tolist()

# print("Loaded embeddings:", all_embeddings.shape)
# print("Loaded file paths:", len(all_files))

Loaded embeddings: (31002, 768)
Loaded file paths: 31002


In [ ]:
# ========== 3. EMBEDDING EXTRACTION (SUPERVISED BACKBONE) ==========
EMBED_FILE = f"{OUT_PREFIX}_embeddings.npy"
PATHS_FILE = f"{OUT_PREFIX}_paths.npy"
if os.path.exists(EMBED_FILE) and os.path.exists(PATHS_FILE):
    all_embeddings = np.load(EMBED_FILE)
    all_files = np.load(PATHS_FILE, allow_pickle=True).tolist()
    print(f"✅ Loaded embeddings: {all_embeddings.shape}")
    print(f"✅ Loaded image paths: {len(all_files)}")
else:
    all_embeddings = []
    all_files = []
    with torch.no_grad():
        for i in tqdm(range(0, len(image_paths), BATCH_SIZE)):
            batch_paths = image_paths[i:i+BATCH_SIZE]
            images = []
            for path in batch_paths:
                try:
                    img = Image.open(path).convert("RGB")
                    images.append(img)
                    all_files.append(path)
                except Exception as e:
                    print(f"Skip: {path} ({e})")
            if not images:
                continue
            inputs = processor(images=images, return_tensors="pt").to(device)
            outputs = model(**inputs, output_hidden_states=True)
            # outputs.hidden_states is a tuple: (layer_0, ..., layer_N)
            # Use the last layer's [CLS] token or mean-pool all tokens (recommended for ViT)
            last_hidden = outputs.hidden_states[-1]  # [batch, seq, dim]
            feats = last_hidden.mean(dim=1).cpu().numpy()
            all_embeddings.append(feats)

    all_embeddings = np.vstack(all_embeddings)
    assert all_embeddings.shape[0] == len(all_files), "Some images were skipped unexpectedly."
    print("✅ Extracted embeddings shape:", all_embeddings.shape)
    np.save(EMBED_FILE, all_embeddings)
    np.save(PATHS_FILE, np.array(all_files))
    print(f"💾 Saved embeddings and image paths to disk as {EMBED_FILE}, {PATHS_FILE}")

In [10]:
# ========== 4. MINI-BATCH KMEANS CLUSTERING ==========
print("⏳ Starting MiniBatchKMeans clustering...")
start = time.perf_counter()

kmeans = MiniBatchKMeans(
    n_clusters=N_CLUSTERS,
    batch_size=1024,
    max_iter=100,
    n_init=CLUSTER_N_INIT,
    init="random",
    random_state=42,
    verbose=1
)

cluster_labels = kmeans.fit_predict(all_embeddings)

end = time.perf_counter()
print(f"✅ MiniBatchKMeans done in {end - start:.2f} sec")
np.save(f"{OUT_PREFIX}_cluster_labels.npy", cluster_labels)

⏳ Starting MiniBatchKMeans clustering...
Init 1/10 with method random
Inertia for init 1/10: 225331920.0
Init 2/10 with method random
Inertia for init 2/10: 242456064.0
Init 3/10 with method random
Inertia for init 3/10: 183423856.0
Init 4/10 with method random
Inertia for init 4/10: 187563840.0
Init 5/10 with method random
Inertia for init 5/10: 204947136.0
Init 6/10 with method random
Inertia for init 6/10: 179880256.0
Init 7/10 with method random
Inertia for init 7/10: 223376432.0
Init 8/10 with method random
Inertia for init 8/10: 202090608.0
Init 9/10 with method random
Inertia for init 9/10: 183129440.0
Init 10/10 with method random
Inertia for init 10/10: 195172272.0
Minibatch step 1/3027: mean batch inertia: 57913.8828125
Minibatch step 2/3027: mean batch inertia: 35938.33984375, ewa inertia: 35938.33984375
Minibatch step 3/3027: mean batch inertia: 33940.296875, ewa inertia: 35806.352874747
Minibatch step 4/3027: mean batch inertia: 34400.15625, ewa inertia: 35713.462164574375

In [11]:
# ========== 5. CLUSTER VISUALIZATION (t-SNE/UMAP, subsampled) ==========
def visualize_embeddings(embeddings, cluster_labels, method="tsne", out_path="embedding_grid.png", subsample=T_SNE_SUBSAMPLE):
    n_points = embeddings.shape[0]
    if n_points > subsample:
        idx = np.random.choice(n_points, subsample, replace=False)
        emb = embeddings[idx]
        lbl = np.array(cluster_labels)[idx]
    else:
        emb = embeddings
        lbl = np.array(cluster_labels)
    if method == "tsne":
        reducer = TSNE(n_components=2, perplexity=50, random_state=42)
    elif method == "umap" and HAVE_UMAP:
        reducer = umap.UMAP(n_components=2, random_state=42)
    else:
        print("Unknown or unavailable method; using t-SNE.")
        reducer = TSNE(n_components=2, perplexity=50, random_state=42)
    coords = reducer.fit_transform(emb)
    plt.figure(figsize=(10, 8))
    
    for k in range(N_CLUSTERS):
        mask = (lbl == k)
        plt.scatter(coords[mask, 0], coords[mask, 1], s=7, alpha=0.7, label=f"C{k}")
    plt.legend()
    plt.title(f"{V13_MODEL_PATH} SSL {method.upper()} by cluster")
    plt.savefig(out_path)
    print(f"🖼️ Saved {method.upper()} visualization to {out_path}")
    plt.close()

visualize_embeddings(
    all_embeddings, cluster_labels,
    method="tsne",
    out_path=f"{OUT_PREFIX}_tsne.png"
)

🖼️ Saved TSNE visualization to ./V16_unsup_20250604_142640/V16_unsup_20250604_142640_temp_autogluon_images_clusters20_kl0.55_eu98_tsne.png


In [12]:
# ========== 6. EUCLIDEAN OUTLIER DETECTION ==========
distances = np.linalg.norm(all_embeddings - kmeans.cluster_centers_[cluster_labels], axis=1)
distance_threshold = np.percentile(distances, EUCLIDEAN_OUTLIER_PERCENTILE)
euclid_outlier_indices = np.where(distances > distance_threshold)[0]
print(f"Flagged {len(euclid_outlier_indices)} outlier images by Euclidean distance.")

Flagged 621 outlier images by Euclidean distance.


In [13]:
# ========== 7. KL-DIVERGENCE OOD DETECTION ==========
def compute_kl_divergence(feats, cluster_centers, labels):
    kl_scores = []
    for f, lbl in zip(feats, labels):
        p = torch.softmax(torch.tensor(f), dim=0)
        q = torch.softmax(torch.tensor(cluster_centers[lbl]), dim=0)
        kl = torch.sum(p * torch.log((p + 1e-8) / (q + 1e-8))).item()
        kl_scores.append(kl)
    return kl_scores

kl_scores = compute_kl_divergence(all_embeddings, kmeans.cluster_centers_, cluster_labels)
kl_outlier_indices = np.where(np.array(kl_scores) > KL_OOD_THRESHOLD)[0]
print(f"Flagged {len(kl_outlier_indices)} OOD images by KL-divergence.")

Flagged 3019 OOD images by KL-divergence.


In [14]:
# ========== 8. SAVE LABELS/OUTLIERS ==========
out_csv = f"{OUT_PREFIX}_ssl_pseudolabels.csv"
with open(out_csv, "w") as f:
    f.write("path,cluster,euclid_dist,kl,euclid_ood,kl_ood\n")
    for idx, (p, c, dist, kl) in enumerate(zip(all_files, cluster_labels, distances, kl_scores)):
        euclid_flag = int(idx in euclid_outlier_indices)
        kl_flag = int(idx in kl_outlier_indices)
        f.write(f"{p},{c},{dist:.3f},{kl:.3f},{euclid_flag},{kl_flag}\n")
print(f"✅ Saved pseudo-labels and OOD flags to {out_csv}")

✅ Saved pseudo-labels and OOD flags to ./V16_unsup_20250604_142640/V16_unsup_20250604_142640_temp_autogluon_images_clusters20_kl0.55_eu98_ssl_pseudolabels.csv


In [15]:
# ========== 9. PER-CLUSTER REPORT ==========
for c in range(N_CLUSTERS):
    cluster_indices = np.where(cluster_labels == c)[0]
    print(f"Cluster {c}: {len(cluster_indices)} samples")

Cluster 0: 1524 samples
Cluster 1: 2085 samples
Cluster 2: 2374 samples
Cluster 3: 850 samples
Cluster 4: 790 samples
Cluster 5: 1055 samples
Cluster 6: 2324 samples
Cluster 7: 984 samples
Cluster 8: 2694 samples
Cluster 9: 1263 samples
Cluster 10: 1371 samples
Cluster 11: 1509 samples
Cluster 12: 1579 samples
Cluster 13: 658 samples
Cluster 14: 2371 samples
Cluster 15: 1505 samples
Cluster 16: 599 samples
Cluster 17: 1728 samples
Cluster 18: 1071 samples
Cluster 19: 2668 samples


In [16]:
# ========== 10. SAVE OOD/JUNK FILES ==========
OOD_DIR = os.path.join(SAVE_DIR, f"{VERSION_TAG}_ood_flagged")
os.makedirs(OOD_DIR, exist_ok=True)
for idx in set(list(euclid_outlier_indices) + list(kl_outlier_indices)):
    src = all_files[idx]
    try:
        fname = os.path.basename(src)
        os.symlink(src, os.path.join(OOD_DIR, fname))
    except FileExistsError:
        continue

print(f"✅ OOD (junk) images symlinked to {OOD_DIR}")

✅ OOD (junk) images symlinked to ./V16_unsup_20250604_142640/V16_unsup_20250604_142640_ood_flagged


In [17]:
# ========== 11. CLUSTER EXPLORER & CENTROID FACES ==========
N_PER_CENTROID = 40   # Or adjust to your preferred # of centroids per cluster
CLUSTER_BROWSER_ROOT = os.path.join(SAVE_DIR, f"{VERSION_TAG}_cluster_browser")

centroids = np.vstack([
    all_embeddings[cluster_labels == c].mean(axis=0)
    for c in range(N_CLUSTERS)
])

for c in range(N_CLUSTERS):
    cdir = os.path.join(CLUSTER_BROWSER_ROOT, f"cluster_{c:02d}")
    centroids_dir = os.path.join(cdir, "centroid_faces")
    os.makedirs(cdir, exist_ok=True)
    os.makedirs(centroids_dir, exist_ok=True)
    indices = np.where(cluster_labels == c)[0]
    if len(indices) == 0:
        continue
    # 1. Symlink/copy all cluster images
    for idx in indices:
        src = all_files[idx]
        dst = os.path.join(cdir, os.path.basename(src))
        try:
            os.symlink(src, dst)
        except FileExistsError:
            continue
        except OSError:
            shutil.copy(src, dst)
    # 2. Find and symlink/copy centroid images
    c_embeds = all_embeddings[indices]
    dists = np.linalg.norm(c_embeds - centroids[c], axis=1)
    sorted_idx = np.argsort(dists)
    chosen = indices[sorted_idx[:N_PER_CENTROID]]
    for idx in chosen:
        src = all_files[idx]
        dst = os.path.join(centroids_dir, os.path.basename(src))
        try:
            os.symlink(src, dst)
        except FileExistsError:
            continue
        except OSError:
            shutil.copy(src, dst)
    print(f"Cluster {c:02d}: {len(indices)} total, {len(chosen)} centroid faces exported")

print(f"✅ Cluster browser and centroid faces exported to {CLUSTER_BROWSER_ROOT}/cluster_##/ and centroid_faces/")

Cluster 00: 1524 total, 40 centroid faces exported
Cluster 01: 2085 total, 40 centroid faces exported
Cluster 02: 2374 total, 40 centroid faces exported
Cluster 03: 850 total, 40 centroid faces exported
Cluster 04: 790 total, 40 centroid faces exported
Cluster 05: 1055 total, 40 centroid faces exported
Cluster 06: 2324 total, 40 centroid faces exported
Cluster 07: 984 total, 40 centroid faces exported
Cluster 08: 2694 total, 40 centroid faces exported
Cluster 09: 1263 total, 40 centroid faces exported
Cluster 10: 1371 total, 40 centroid faces exported
Cluster 11: 1509 total, 40 centroid faces exported
Cluster 12: 1579 total, 40 centroid faces exported
Cluster 13: 658 total, 40 centroid faces exported
Cluster 14: 2371 total, 40 centroid faces exported
Cluster 15: 1505 total, 40 centroid faces exported
Cluster 16: 599 total, 40 centroid faces exported
Cluster 17: 1728 total, 40 centroid faces exported
Cluster 18: 1071 total, 40 centroid faces exported
Cluster 19: 2668 total, 40 centroid 

In [20]:
# ========== 12. V13 PREDICTION EXTRACTION FOR ALL IMAGES ==========

# Store logits and predict V13 softmax for all images
all_logits = []
all_probs = []
BATCH_SIZE = 32

model.eval()
with torch.no_grad():
    for i in tqdm(range(0, len(all_files), BATCH_SIZE), desc="V13 Prediction Extraction"):
        batch_paths = all_files[i:i+BATCH_SIZE]
        images = []
        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
                images.append(img)
            except Exception as e:
                print(f"Skip: {path} ({e})")
        if not images:
            continue
        inputs = processor(images=images, return_tensors="pt").to(model.device)
        outputs = model(**inputs)
        logits = outputs.logits.cpu().numpy()
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
        all_logits.append(logits)
        all_probs.append(probs)

all_logits = np.vstack(all_logits)
all_probs = np.vstack(all_probs)
np.save(os.path.join(SAVE_DIR, "v13_logits.npy"), all_logits)
np.save(os.path.join(SAVE_DIR, "v13_probs.npy"), all_probs)
print("✅ Saved V13 logits/probabilities for all images.")

V13 Prediction Extraction: 100%|██████████████| 969/969 [22:44<00:00,  1.41s/it]

✅ Saved V13 logits/probabilities for all images.


In [44]:
# ========== 13. LABEL CONSISTENCY HEATMAP ==========

all_preds = np.argmax(all_probs, axis=1)
n_clusters = np.max(cluster_labels) + 1
n_labels = len(LABEL_NAMES)  # Or len(id2label)

cluster_label_counts = np.zeros((n_clusters, n_labels), dtype=int)
for cl, lab in zip(cluster_labels, V13_preds):
    cluster_label_counts[cl, lab] += 1

cluster_label_perc = cluster_label_counts / cluster_label_counts.sum(axis=1, keepdims=True)

plt.figure(figsize=(12, 7))
sns.heatmap(
    cluster_label_perc, annot=True, fmt=".2f",
    xticklabels=LABEL_NAMES,
    yticklabels=[f"C{k}" for k in range(n_clusters)]
)
plt.xlabel("V13 Predicted Label")
plt.ylabel("Cluster")
plt.title("Cluster Label Consistency Heatmap")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, f"{VERSION_TAG}_cluster_label_heatmap.png"))
plt.close()

In [28]:
# ========== 14. CLUSTER ENTROPY BARPLOT ==========
entropy_per_image = entropy(all_probs, axis=1)
avg_entropy_per_cluster = [
    np.mean(entropy_per_image[cluster_labels == k])
    for k in range(n_clusters)
]

plt.figure(figsize=(10, 4))
plt.bar(range(n_clusters), avg_entropy_per_cluster)
plt.xlabel("Cluster")
plt.ylabel("Avg Entropy")
plt.title("Mean Prediction Entropy per Cluster")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, f"{VERSION_TAG}_cluster_entropy_barplot.png"))
plt.close()

In [37]:
# ========== 15. CLUSTER REFINEMENT: REMOVE LOW-CONFIDENCE SAMPLES ==========
CONFIDENCE_THRESH = 0.20

pred_confidences = all_probs.max(axis=1)
keep_mask = pred_confidences >= CONFIDENCE_THRESH

# Filter arrays
refined_embeddings = all_embeddings[keep_mask]
refined_files = [all_files[i] for i, keep in enumerate(keep_mask) if keep]
refined_cluster_labels = cluster_labels[keep_mask]
refined_probs = all_probs[keep_mask]
refined_preds = np.argmax(refined_probs, axis=1)
print(f"Retained {len(refined_files)}/{len(all_files)} images with confidence >= {CONFIDENCE_THRESH:.2f}")

Retained 1480/31002 images with confidence >= 0.20


In [38]:
# ========== 16. KMEANS ON PCA-REDUCED EMBEDDINGS ==========
PCA_DIM = 64
pca = PCA(n_components=PCA_DIM, random_state=42)
pca_embeds = pca.fit_transform(refined_embeddings)

kmeans_pca = MiniBatchKMeans(
    n_clusters=N_CLUSTERS,
    batch_size=1024,
    max_iter=100,
    n_init=CLUSTER_N_INIT,
    init="random",
    random_state=42,
    verbose=1
)
refined_cluster_labels_pca = kmeans_pca.fit_predict(pca_embeds)
np.save(os.path.join(SAVE_DIR, "refined_pca_cluster_labels.npy"), refined_cluster_labels_pca)
print("✅ KMeans on PCA-reduced, confidence-filtered embeddings complete.")

Init 1/10 with method random
Inertia for init 1/10: 38619104.0
Init 2/10 with method random
Inertia for init 2/10: 40644380.0
Init 3/10 with method random
Inertia for init 3/10: 38730072.0
Init 4/10 with method random
Inertia for init 4/10: 39801012.0
Init 5/10 with method random
Inertia for init 5/10: 40222928.0
Init 6/10 with method random
Inertia for init 6/10: 39472344.0
Init 7/10 with method random
Inertia for init 7/10: 39328016.0
Init 8/10 with method random
Inertia for init 8/10: 39305020.0
Init 9/10 with method random
Inertia for init 9/10: 38297216.0
Init 10/10 with method random
Inertia for init 10/10: 38536700.0
Minibatch step 1/144: mean batch inertia: 25759.5625
Minibatch step 2/144: mean batch inertia: 16651.068359375, ewa inertia: 16651.068359375
Minibatch step 3/144: mean batch inertia: 16800.64453125, ewa inertia: 16800.64453125
Minibatch step 4/144: mean batch inertia: 16487.076171875, ewa inertia: 16487.076171875
Minibatch step 5/144: mean batch inertia: 16046.20703

In [39]:
# ========== 17. SUMMARY/EXPORT/README ==========
summary_path = os.path.join(SAVE_DIR, "run_summary.md")
with open(summary_path, "w") as f:
    f.write(f"# V17_unsup Run Summary\n")
    f.write(f"Date: {datetime.datetime.now()}\n\n")
    f.write(f"Model: {V13_MODEL_PATH}\n")
    f.write(f"Image Dir: {IMAGE_DIR}\n")
    f.write(f"Num Images: {len(all_files)}\n")
    f.write(f"Num Clusters: {N_CLUSTERS}\n")
    f.write(f"Confidence threshold: {CONFIDENCE_THRESH}\n")
    f.write(f"Kept after filtering: {len(refined_files)} images\n\n")

    # (Add confusion/purity stats here as you compute them)
    f.write("## Cluster Sizes (after refinement):\n")
    unique, counts = np.unique(refined_cluster_labels, return_counts=True)
    for cid, sz in zip(unique, counts):
        f.write(f"- Cluster {cid}: {sz} images\n")
    f.write("\n")
    # If you have cluster entropy/purity, include those stats as well.

    f.write("\n## Next Steps\n- Review high-entropy clusters\n- Use pure clusters for semi-supervised fine-tuning\n")
print(f"✅ Summary file written to {summary_path}")

✅ Summary file written to ./V16_unsup_20250604_142640/run_summary.md


In [41]:
# ========== 18. SAVE CONFIDENCE HISTOGRAM ==========
plt.figure(figsize=(8, 5))
plt.hist(pred_confidences, bins=40, color='skyblue', edgecolor='black')
plt.xlabel("V13 max softmax confidence")
plt.ylabel("Number of images")
plt.title("V13 model confidence histogram")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, f"{VERSION_TAG}_v13_confidence_histogram.png"))
plt.close()
print(f"✅ Saved confidence histogram to {SAVE_DIR}")

✅ Saved confidence histogram to ./V16_unsup_20250604_142640


In [42]:
# ========== 19. SHOW EXTREMES: HIGH/LOW CONFIDENCE IMAGE SAMPLES ==========
def show_images_by_confidence(image_paths, confidences, top_n=5):
    sorted_idx = np.argsort(confidences)
    fig, axes = plt.subplots(2, top_n, figsize=(top_n*3, 6))
    # Lowest confidence
    for i in range(top_n):
        img = Image.open(image_paths[sorted_idx[i]]).convert("RGB")
        axes[0, i].imshow(img)
        axes[0, i].set_title(f"Low conf: {confidences[sorted_idx[i]]:.2f}")
        axes[0, i].axis('off')
    # Highest confidence
    for i in range(top_n):
        img = Image.open(image_paths[sorted_idx[-(i+1)]]).convert("RGB")
        axes[1, i].imshow(img)
        axes[1, i].set_title(f"High conf: {confidences[sorted_idx[-(i+1)]]:.2f}")
        axes[1, i].axis('off')
    plt.suptitle("V13 confidence extremes (top: lowest, bottom: highest)")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f"{VERSION_TAG}_confidence_extremes.png"))
    plt.close()
    print("✅ Saved high/low confidence sample image grid.")

show_images_by_confidence(all_files, pred_confidences, top_n=5)

✅ Saved high/low confidence sample image grid.


In [45]:
# ========== 20. CLUSTER CONFIDENCE & ENTROPY DIAGNOSTICS ==========
cluster_diag = []
for c in range(N_CLUSTERS):
    indices = np.where(cluster_labels == c)[0]
    if len(indices) == 0: continue
    confs = pred_confidences[indices]
    entrs = entropy(all_probs[indices], axis=1)
    preds = all_preds[indices]
    dominant = np.argmax(np.bincount(preds))
    dominant_label = LABEL_NAMES[dominant]
    purity = (preds == dominant).sum() / len(preds)
    cluster_diag.append({
        "cluster": c,
        "size": len(indices),
        "mean_conf": np.mean(confs),
        "median_conf": np.median(confs),
        "mean_entropy": np.mean(entrs),
        "dominant_label": dominant_label,
        "purity": purity
    })

df_diag = pd.DataFrame(cluster_diag)
df_diag.sort_values("purity", ascending=False, inplace=True)
df_diag.to_csv(os.path.join(SAVE_DIR, f"{VERSION_TAG}_cluster_diagnostics.csv"), index=False)
print(df_diag)

# Highlight clusters for curation if purity < 0.8 or mean_conf < 0.4
low_purity = df_diag[df_diag.purity < 0.8]
low_conf = df_diag[df_diag.mean_conf < 0.4]
print("Clusters flagged for curation (purity < 0.8):\n", low_purity)
print("Clusters flagged for curation (mean_conf < 0.4):\n", low_conf)

    cluster  size  mean_conf  median_conf  mean_entropy dominant_label  \
3         3   850   0.159202     0.159180      2.071729       surprise   
1         1  2085   0.160253     0.160141      2.071323       surprise   
19       19  2668   0.161803     0.161622      2.070526       surprise   
2         2  2374   0.165737     0.165109      2.068169       surprise   
17       17  1728   0.147811     0.147790      2.071930        sadness   
14       14  2371   0.174562     0.172189      2.060905       surprise   
9         9  1263   0.148008     0.147792      2.071489        sadness   
11       11  1509   0.148457     0.147948      2.071152        sadness   
4         4   790   0.165656     0.164423      2.052084    questioning   
16       16   599   0.186200     0.185668      2.000031          anger   
12       12  1579   0.144430     0.144077      2.072072          anger   
6         6  2324   0.186145     0.187796      2.038842        sadness   
15       15  1505   0.160233     0.157

In [46]:
# ========== 21. EXPORT HIGH-PURITY CLUSTERS FOR MANUAL REVIEW/INJECTION ==========
EXPORT_PURITY = 0.9  # Only clusters with at least 90% purity will be exported
for _, row in df_diag.iterrows():
    if row['purity'] >= EXPORT_PURITY and row['size'] > 10:
        c = int(row['cluster'])
        indices = np.where(cluster_labels == c)[0]
        export_dir = os.path.join(SAVE_DIR, f"cluster_{c:02d}_pure_{row['dominant_label']}")
        os.makedirs(export_dir, exist_ok=True)
        for idx in indices:
            src = all_files[idx]
            dst = os.path.join(export_dir, os.path.basename(src))
            try:
                os.symlink(src, dst)
            except FileExistsError:
                continue
            except OSError:
                import shutil
                shutil.copy(src, dst)
        print(f"Exported cluster {c:02d} ({row['dominant_label']}) to {export_dir}")

Exported cluster 03 (surprise) to ./V16_unsup_20250604_142640/cluster_03_pure_surprise
Exported cluster 01 (surprise) to ./V16_unsup_20250604_142640/cluster_01_pure_surprise
Exported cluster 19 (surprise) to ./V16_unsup_20250604_142640/cluster_19_pure_surprise
Exported cluster 02 (surprise) to ./V16_unsup_20250604_142640/cluster_02_pure_surprise
Exported cluster 17 (sadness) to ./V16_unsup_20250604_142640/cluster_17_pure_sadness
Exported cluster 14 (surprise) to ./V16_unsup_20250604_142640/cluster_14_pure_surprise


In [47]:
# ========== 22. CLUSTER-WISE CONFIDENCE SUMMARY AND PLOT ==========
# (included in diagnostics but as a stand-alone plot)
plt.figure(figsize=(10, 4))
plt.bar(df_diag['cluster'], df_diag['mean_conf'])
plt.xlabel("Cluster")
plt.ylabel("Mean V13 Confidence")
plt.title("Mean Confidence per Cluster")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, f"{VERSION_TAG}_mean_conf_per_cluster.png"))
plt.close()
print(f"✅ Saved per-cluster confidence plot to {SAVE_DIR}")

✅ Saved per-cluster confidence plot to ./V16_unsup_20250604_142640


In [48]:
# ========== 23. CLUSTER PURIFICATION: EXPORT TOP-N HIGH-CONFIDENCE FACES PER CLUSTER ==========
N_TOP = 20  # how many high-confidence faces per cluster to copy/symlink

for c in range(N_CLUSTERS):
    indices = np.where(cluster_labels == c)[0]
    if len(indices) == 0:
        continue
    confs = pred_confidences[indices]
    sorted_idx = indices[np.argsort(confs)[::-1]]  # Descending
    chosen = sorted_idx[:N_TOP]
    cluster_dir = os.path.join(SAVE_DIR, f"cluster_{c:02d}_top_conf")
    os.makedirs(cluster_dir, exist_ok=True)
    for idx in chosen:
        src = all_files[idx]
        dst = os.path.join(cluster_dir, os.path.basename(src))
        try:
            os.symlink(src, dst)
        except FileExistsError:
            continue
        except OSError:
            import shutil
            shutil.copy(src, dst)
    print(f"Cluster {c:02d}: exported {len(chosen)} highest-confidence faces")
print(f"✅ Cluster purification complete (see *_top_conf folders in {SAVE_DIR})")

Cluster 00: exported 20 highest-confidence faces
Cluster 01: exported 20 highest-confidence faces
Cluster 02: exported 20 highest-confidence faces
Cluster 03: exported 20 highest-confidence faces
Cluster 04: exported 20 highest-confidence faces
Cluster 05: exported 20 highest-confidence faces
Cluster 06: exported 20 highest-confidence faces
Cluster 07: exported 20 highest-confidence faces
Cluster 08: exported 20 highest-confidence faces
Cluster 09: exported 20 highest-confidence faces
Cluster 10: exported 20 highest-confidence faces
Cluster 11: exported 20 highest-confidence faces
Cluster 12: exported 20 highest-confidence faces
Cluster 13: exported 20 highest-confidence faces
Cluster 14: exported 20 highest-confidence faces
Cluster 15: exported 20 highest-confidence faces
Cluster 16: exported 20 highest-confidence faces
Cluster 17: exported 20 highest-confidence faces
Cluster 18: exported 20 highest-confidence faces
Cluster 19: exported 20 highest-confidence faces
✅ Cluster purificati

In [49]:
# ========== 24. OPTIONAL: TEMPERATURE SCALING FOR CALIBRATION ==========
# (Requires a clean, labeled validation set with logits/labels)
def temperature_scale(logits, labels):
    temperature = nn.Parameter(torch.ones(1) * 1.5)
    optimizer = optim.LBFGS([temperature], lr=0.01, max_iter=50)
    nll_criterion = nn.CrossEntropyLoss()

    def eval():
        optimizer.zero_grad()
        loss = nll_criterion(logits / temperature, labels)
        loss.backward()
        return loss

    optimizer.step(eval)
    return temperature.item()

# Example usage (with val_logits, val_labels as tensors):
# T_opt = temperature_scale(val_logits, val_labels)
# scaled_probs = torch.softmax(torch.tensor(all_logits) / T_opt, dim=1).numpy()
# Use scaled_probs for confidence diagnostics if desired.

In [50]:
# ========== 25. SUMMARY/EXPORT/README (ALREADY IN YOUR PIPELINE, RECAP) ==========
summary_path = os.path.join(SAVE_DIR, "run_summary.md")
with open(summary_path, "w") as f:
    f.write(f"# V17_unsup Run Summary\n")
    f.write(f"Date: {datetime.datetime.now()}\n\n")
    f.write(f"Model: {V13_MODEL_PATH}\n")
    f.write(f"Image Dir: {IMAGE_DIR}\n")
    f.write(f"Num Images: {len(all_files)}\n")
    f.write(f"Num Clusters: {N_CLUSTERS}\n")
    f.write(f"Confidence threshold: {CONFIDENCE_THRESH}\n")
    f.write(f"Kept after filtering: {len(refined_files)} images\n\n")

    # (Add confusion/purity stats here as you compute them)
    f.write("## Cluster Sizes (after refinement):\n")
    unique, counts = np.unique(refined_cluster_labels, return_counts=True)
    for cid, sz in zip(unique, counts):
        f.write(f"- Cluster {cid}: {sz} images\n")
    f.write("\n")
    # If you have cluster entropy/purity, include those stats as well.

    f.write("\n## Next Steps\n- Review high-entropy clusters\n- Use pure clusters for semi-supervised fine-tuning\n")
print(f"✅ Summary file written to {summary_path}")

✅ Summary file written to ./V16_unsup_20250604_142640/run_summary.md
